<a href="https://colab.research.google.com/github/Mihir-Panjikar/Legal_Document_Simplication_System/blob/dev/Notebook/Llama_legal_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%reset -f
from IPython.display import clear_output
clear_output()
import torch
torch.cuda.empty_cache()

In [2]:
!pip install --upgrade huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.1/468.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1


In [3]:
from google.colab import userdata
read_token = userdata.get('read_hugging_face')

In [4]:
!huggingface-cli login --token $read_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `Read_access` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Read_access`


In [5]:
!pip install --upgrade transformers
!pip install torch
!pip install --upgrade bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 57.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "meta-llama/Llama-3.3-70B-Instruct"

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

quantized_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

def format_messages(messages):
    formatted = ""
    for msg in messages:
        formatted += f"{msg['role'].capitalize()}: {msg['content']}\n\n"
    return formatted

input_text = format_messages([
    {"role": "system", "content": """You are an expert in summarization and legal document simplification. Your task is to summarize the following agreement in a way that is easy for a layperson to understand. The summary should include all key details while using simple, clear language and relevant real-life examples where needed. Ensure that no critical information is lost.

Key requirements for the summary:

Use plain English, avoiding complex legal terms.
Provide relevant examples to make the agreement more understandable.
Maintain all important details, such as duration, termination clauses, liability, dispute resolution, and governing law.
Ensure the summary remains concise but does not miss crucial points."""},
    {"role": "user", "content": """This Agreement shall commence on the Effective Date and shall continue for a period of five (5) years unless earlier terminated in accordance with the provisions set forth herein. Either Party may terminate this Agreement upon thirty (30) days prior written notice to the other Party in the event of a material breach that remains uncured for a period of fifteen (15) days following written notice of such breach. In the event of termination, both Parties shall fulfill any obligations incurred prior to the effective date of termination. Furthermore, neither Party shall be liable to the other for any indirect, consequential, or punitive damages arising out of or in connection with this Agreement, except in cases of willful misconduct or gross negligence. Any disputes arising under this Agreement shall be resolved through binding arbitration in accordance with the rules of the American Arbitration Association, and the prevailing Party shall be entitled to recover reasonable attorneys’ fees and costs incurred in connection with such arbitration. This Agreement shall be governed by and construed in accordance with the laws of the State of New York, without regard to its conflict of law principles. If any provision of this Agreement is found to be invalid or unenforceable, the remaining provisions shall continue in full force and effect. No modification, amendment, or waiver of any provision of this Agreement shall be effective unless in writing and signed by both Parties. This Agreement constitutes the entire agreement between the Parties with respect to the subject matter herein and supersedes all prior and contemporaneous agreements, representations, and understandings, whether written or oral."""}
])

input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

output = quantized_model.generate(**input_ids, max_new_tokens=256)

print(tokenizer.decode(output[0], skip_special_tokens=True))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/59.6k [00:00<?, ?B/s]

model-00001-of-00030.safetensors:   0%|          | 0.00/4.58G [00:00<?, ?B/s]

model-00002-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

...

model-00011-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00012-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00013-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00014-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00015-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]